# Creating and Valuing a CDS Index Option

This notebook does the valuation of an option on a CDS index using Anderson's method and an Adjusted Black Method. For details see Modelling Singlename and Multiname Credit Derivatives by D.O'Kane.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.finutils import *
from financepy.products.libor import *
from financepy.products.credit import *

####################################################################
# FINANCEPY BETA Version 0.180 - This build:  22 Sep 2020 at 22:40 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#    For bug reports and comments - allmypythonprojects@gmail.com  #
####################################################################



In [3]:
tradeDate = FinDate(1, 8, 2007)
stepInDate = tradeDate.addDays(1)
valuationDate = stepInDate
settlementDate = stepInDate

## Build Libor Curve

In [4]:
depos = []
dcType = FinDayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FinFrequencyTypes.SEMI_ANNUAL
swapType = FinLiborSwapTypes.PAYER
swap1 = FinLiborSwap(settlementDate,"1Y",swapType,0.0502,fixedFreq,dcType)
swap2 = FinLiborSwap(settlementDate,"2Y",swapType,0.0502,fixedFreq,dcType)
swap3 = FinLiborSwap(settlementDate,"3Y",swapType,0.0501,fixedFreq,dcType)
swap4 = FinLiborSwap(settlementDate,"4Y",swapType,0.0502,fixedFreq,dcType)
swap5 = FinLiborSwap(settlementDate,"5Y",swapType,0.0501,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5]

liborCurve = FinLiborCurve(settlementDate, depos, [], swaps)

In [5]:
cdsMaturity3Y = tradeDate.nextCDSDate(36)
cdsMaturity5Y = tradeDate.nextCDSDate(60)
cdsMaturity7Y = tradeDate.nextCDSDate(84)
cdsMaturity10Y = tradeDate.nextCDSDate(120)

## Load the Underlying CDS Index Portfolio

In [6]:
f = open('.//data//CDX_NA_IG_S7_SPREADS.csv', 'r')
data = f.readlines()
issuerCurves = []

numCredits = len(data) - 1  # The file has a header

for row in data[1:]:
    splitRow = row.split(",")
    creditName = splitRow[0]
    spd3Y = float(splitRow[1]) / 10000.0
    spd5Y = float(splitRow[2]) / 10000.0
    spd7Y = float(splitRow[3]) / 10000.0
    spd10Y = float(splitRow[4]) / 10000.0
    recoveryRate = float(splitRow[5])

    cds3Y = FinCDS(stepInDate, cdsMaturity3Y, spd3Y)
    cds5Y = FinCDS(stepInDate, cdsMaturity5Y, spd5Y)
    cds7Y = FinCDS(stepInDate, cdsMaturity7Y, spd7Y)
    cds10Y = FinCDS(stepInDate, cdsMaturity10Y, spd10Y)
    cdsContracts = [cds3Y, cds5Y, cds7Y, cds10Y]

    issuerCurve = FinCDSCurve(valuationDate, cdsContracts, liborCurve, recoveryRate)
    issuerCurves.append(issuerCurve)

## Term Structure of CDS Index Market

We now set up the term structure of CDS Index trades and start by setting up the maturity dates of the index

In [7]:
indexMaturity3Y = FinDate(20,12,2009)
indexMaturity5Y = FinDate(20,12,2011)
indexMaturity7Y = FinDate(20,12,2013)
indexMaturity10Y = FinDate(20,12,2016)

Now we set their upfronts and coupons

In [8]:
indexUpfronts = [0.0, 0.0, 0.0, 0.0]
indexMaturityDates = [indexMaturity3Y, indexMaturity5Y, indexMaturity7Y, indexMaturity10Y]
indexRecovery = 0.40

## Setting up the Index Option

In [9]:
expiryDate = FinDate(1,2,2008)
maturityDate = FinDate(20,12,2011)
notional = 10000
volatility = 0.50
indexCoupon = 0.004
tolerance = 1e-6

## Comparison of Anderson versus Adjusted Black Model

In [10]:
print("STRIKE    INDEX      PAYER    RECEIVER      G(K)       X         EXPH       ABPAY       ABREC")

for index in np.linspace(20, 60, 5):
    
    # Create a flat CDS index curve
    cdsContracts = []
    for dt in indexMaturityDates:
        cds = FinCDS(valuationDate, dt, index / 10000.0)
        cdsContracts.append(cds)

    # Build the flat CDS index curve
    indexCurve = FinCDSCurve(valuationDate, cdsContracts, liborCurve, indexRecovery)

    # Now we need to set up the underlying curves and adjust them
    indexSpreads = [index / 10000.0] * 4

    indexPortfolio = FinCDSIndexPortfolio()
    adjustedIssuerCurves = indexPortfolio.hazardRateAdjustIntrinsic(valuationDate, 
                                                                    issuerCurves, 
                                                                    indexSpreads, 
                                                                    indexUpfronts,
                                                                    indexMaturityDates, 
                                                                    indexRecovery, 
                                                                    tolerance)        
        
    for strike in np.linspace(20, 60, 5):    

        option = FinCDSIndexOption(expiryDate,
                                   maturityDate, 
                                   indexCoupon, 
                                   strike / 10000.0, 
                                   notional)
        
        v_pay_1, v_rec_1, strikeValue, mu, expH = option.valueAnderson(valuationDate, 
                                                                       adjustedIssuerCurves, 
                                                                       indexRecovery,
                                                                       volatility)

        v_pay_2, v_rec_2 = option.valueAdjustedBlack(valuationDate,
                                                     indexCurve, 
                                                     indexRecovery, 
                                                     liborCurve,
                                                     volatility)
            
        print("%6.3f   %6.3f   %8.3f   %8.3f   %8.3f   %8.2f   %8.2f   %8.2f   %8.2f" % 
              (strike, index, v_pay_1, v_rec_1, strikeValue, mu, expH, v_pay_2, v_rec_2 ))

STRIKE    INDEX      PAYER    RECEIVER      G(K)       X         EXPH       ABPAY       ABREC
20.000   20.000     16.106      6.194    -70.746      22.89     -60.59      16.13       6.10
30.000   20.000      4.033     28.800    -35.261      22.87     -60.65       4.03      28.65
40.000   20.000      0.879     60.107      0.000      22.85     -60.71       0.89      59.89
50.000   20.000      0.185     93.656     35.038      22.84     -60.77       0.20      93.29
60.000   20.000      0.042    127.540     69.855      22.82     -60.83       0.05     126.96
20.000   30.000     50.393      0.880    -70.746      34.38     -20.00      50.46       0.83
30.000   30.000     24.055      9.191    -35.261      34.37     -20.03      24.09       9.14
40.000   30.000      9.738     29.306      0.000      34.36     -20.06       9.82      29.27
50.000   30.000      3.641     57.423     35.038      34.35     -20.09       3.70      57.28
60.000   30.000      1.316     89.095     69.855      34.34     -20.1

In [11]:
print(option)

OBJECT TYPE: FinCDSIndexOption
EXPIRY DATE: FRI 01 FEB 2008
MATURITY DATE: TUE 20 DEC 2011
INDEX COUPON: 40.0bp
NOTIONAL: 10000
LONG PROTECTION: True
FREQUENCY: FinFrequencyTypes.QUARTERLY
DAYCOUNT: FinDayCountTypes.ACT_360
CALENDAR: FinCalendarTypes.WEEKEND
BUSDAYRULE: FinBusDayAdjustTypes.FOLLOWING
DATEGENRULE: FinDateGenRuleTypes.BACKWARD

